In [1]:
import ciropt as co
import cvxpy as cp

import PEPit
import PEPit.functions as pep_func
import ciropt.function as co_func
import sympy as sp

In [2]:
L_smooth = 1.
mu = 0.1 #0

Capacitance = 2.
Inductance = 1.
R = 1.

solver = "ipopt"
# solver = "ipopt_qcqp"
# solver = "ipopt_qcqp_matrix" 

# Ciropt problem

In [3]:
def accelerated_gradient_circuit(mu, L_smooth, params=None): 
    if params is not None:
        # verification mode: PEP
        problem = PEPit.PEP()
        package = pep_func
        h, alpha, beta, eta, rho, C, invC, L, invL, R = params["h"], params["alpha"], params["beta"], params["eta"], \
                                    params["rho"], params["C"], params["invC"], params["L"], params["invL"], params["R"]
    else:
        # Ciropt mode
        problem = co.CircuitOpt()
        package = co_func
        C, R, L = sp.symbols('C'), sp.symbols('R'), sp.symbols('L')
        invC, invL = sp.symbols("invC"), sp.symbols("invL")
        problem.discretization_params = sorted(problem.discretization_params + 
                                               ["C", "invC", "L", "invL", "R"])
        h, alpha, beta, eta, rho = problem.h, problem.alpha, problem.beta, problem.eta, problem.rho

    func = co.define_function(problem, mu, L_smooth, package)
    x_star, y_star, f_star = func.stationary_point(return_gradient_and_function_value=True)

    v_C_1 = problem.set_initial_point()
    i_L_1 = problem.set_initial_point()
    x_1 = R * i_L_1 + v_C_1
    y_1, f_1 = func.oracle(x_1)

    i_L_1p5 = i_L_1 + (alpha * h * invL) * (v_C_1 - (x_1 - R * y_1)) 
    v_C_1p5 = v_C_1 - (alpha * h * invC) * y_1 
    x_1p5 = R * i_L_1p5 + v_C_1p5
    y_1p5, f_1p5 = func.oracle(x_1p5)

    i_L_2 = i_L_1 + (beta * h * invL) * (v_C_1 - (x_1 - R * y_1)) + \
                    ((1 - beta) * h * invL) * (v_C_1p5 - (x_1p5 - R * y_1p5))
    v_C_2 = v_C_1 - (beta * h * invC) * y_1 - ((1 - beta) * h * invC) * y_1p5  
    x_2 = R * i_L_2 + v_C_2
    y_2, f_2 = func.oracle(x_2)

    E_1 = (C/2) * (v_C_1 - x_star)**2 + (L/2) * (i_L_1 - y_star) ** 2
    E_2 = (C/2) * (v_C_2 - x_star)**2 + (L/2) * (i_L_2 - y_star) ** 2
    Delta_1 = rho * R * (y_1 - i_L_1)**2 + eta * (f_1 - f_star)
    problem.set_performance_metric(E_2 - (E_1 - Delta_1))
    return problem


In [5]:
problem = accelerated_gradient_circuit(mu, L_smooth)
problem.obj = problem.eta + problem.rho

bounds = {"C":{"ub": 10, "lb":0.5},
          "R":{"ub": 10, "lb":0.5},
          "L":{"ub": 10, "lb":0.5}}

res, model, sp_exp = problem.solve(solver=solver, bounds=bounds, verbose=False, debug=True)

dim_G=6, dim_F=4
Ipopt total # of variables = 374
Actual # of variables = 49


In [6]:
res

{'C': 10.000000098593643,
 'L': 10.00000004985383,
 'R': 0.49999999005637524,
 'alpha': 0.25662344259673525,
 'beta': -0.17464682307232068,
 'eta': 2.78344092932915,
 'h': 10.644399118460987,
 'invC': 0.09999999901406355,
 'invL': 0.09999999950146175,
 'rho': 5.832952935283037}

# PEP verification

In [9]:
problem = accelerated_gradient_circuit(mu, L_smooth, params=res)
diff = problem.solve(verbose=0, solver=cp.MOSEK)
print(f"{diff=}")

diff=7.945997673989319e-09


/Users/parshakova.tanya/opt/anaconda3/envs/circuits/lib/python3.10/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
